In [ ]:
import openpyxl
import os

def copy_cell_styles(source_cell, target_cell):
    """Copy the style attributes from the source cell to the target cell."""
    target_cell.font = openpyxl.styles.Font(name=source_cell.font.name,
                                            size=source_cell.font.size,
                                            bold=source_cell.font.bold,
                                            italic=source_cell.font.italic,
                                            vertAlign=source_cell.font.vertAlign,
                                            underline=source_cell.font.underline,
                                            strike=source_cell.font.strike,
                                            color=source_cell.font.color)
    
    target_cell.fill = openpyxl.styles.PatternFill(fill_type=source_cell.fill.fill_type,
                                                   start_color=source_cell.fill.start_color,
                                                   end_color=source_cell.fill.end_color)
    
    target_cell.border = openpyxl.styles.Border(left=source_cell.border.left,
                                                right=source_cell.border.right,
                                                top=source_cell.border.top,
                                                bottom=source_cell.border.bottom)
    
    target_cell.alignment = openpyxl.styles.Alignment(horizontal=source_cell.alignment.horizontal,
                                                      vertical=source_cell.alignment.vertical,
                                                      text_rotation=source_cell.alignment.text_rotation,
                                                      wrap_text=source_cell.alignment.wrap_text,
                                                      shrink_to_fit=source_cell.alignment.shrink_to_fit,
                                                      indent=source_cell.alignment.indent)
    
    target_cell.number_format = source_cell.number_format
    target_cell.protection = openpyxl.styles.Protection(locked=source_cell.protection.locked,
                                                        hidden=source_cell.protection.hidden)

def copy_dimensions(source_sheet, target_sheet):
    """Copy row heights and column widths from source sheet to target sheet."""
    for col_letter, col_dim in source_sheet.column_dimensions.items():
        target_sheet.column_dimensions[col_letter].width = col_dim.width
    
    for row_idx, row_dim in source_sheet.row_dimensions.items():
        target_sheet.row_dimensions[row_idx].height = row_dim.height

def copy_merged_cells(source_sheet, target_sheet):
    """Copy merged cell ranges from source sheet to target sheet."""
    for merged_range in source_sheet.merged_cells.ranges:
        target_sheet.merge_cells(str(merged_range))

def copy_data_validations(source_sheet, target_sheet):
    """Copy data validation rules from source sheet to target sheet."""
    for data_validation in source_sheet.data_validations.dataValidation:
        # Create a new data validation object with the same settings
        dv_new = openpyxl.worksheet.datavalidation.DataValidation(
            type=data_validation.type,
            formula1=data_validation.formula1,
            formula2=data_validation.formula2,
            allow_blank=data_validation.allowBlank,
            operator=data_validation.operator,
            showErrorMessage=data_validation.showErrorMessage,
            showInputMessage=data_validation.showInputMessage,
            promptTitle=data_validation.promptTitle,
            prompt=data_validation.prompt,
            errorTitle=data_validation.errorTitle,
            error=data_validation.error,
        )
        dv_new.ranges = data_validation.ranges  # Copy the ranges
        target_sheet.add_data_validation(dv_new)

def split_excel(input_file, output_folder, header_row, split_column):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        wb = openpyxl.load_workbook(input_file)
        sheet = wb.active

        header_rows = list(sheet.iter_rows(min_row=1, max_row=header_row))
        split_column_index = split_column - 1

        split_column_values = []
        for row in sheet.iter_rows(min_row=header_row + 1):
            cell_value = row[split_column_index].value
            if cell_value not in split_column_values:
                split_column_values.append(cell_value)

        for value in split_column_values:
            filename = os.path.join(output_folder, f"2023、2024年已毕业但至今未就业人员情况表_{value}.xlsx")
            print("Generating file:", filename)

            wb_new = openpyxl.Workbook()
            sheet_new = wb_new.active
            sheet_new.title = "Sheet1"

            copy_dimensions(sheet, sheet_new)
            copy_merged_cells(sheet, sheet_new)
            copy_data_validations(sheet, sheet_new)

            for row_idx, header_row_cells in enumerate(header_rows, start=1):
                for col_idx, source_cell in enumerate(header_row_cells, start=1):
                    target_cell = sheet_new.cell(row=row_idx, column=col_idx, value=source_cell.value)
                    copy_cell_styles(source_cell, target_cell)

            row_new = header_row + 1
            for row in sheet.iter_rows(min_row=header_row + 1):
                if row[split_column_index].value == value:
                    for col_idx, source_cell in enumerate(row, start=1):
                        target_cell = sheet_new.cell(row=row_new, column=col_idx, value=source_cell.value)
                        copy_cell_styles(source_cell, target_cell)
                    row_new += 1

            wb_new.save(filename)
            print("File saved:", filename)

        print("All files split successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file = r'/Users/gejundeng/Desktop/附件1-4.xlsx'
output_folder = r'/Users/gejundeng/Desktop/拆分文件夹'
header_row = 4
split_column = 12

split_excel(input_file, output_folder, header_row, split_column)